In [7]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'

firefox = webdriver.Firefox(capabilities=firefox_capabilities, executable_path=geckoPath)
driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')


#firefox.get('https://www.crunchbase.com/organization/apple')
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(3)
def extract_company_techcrunch_2drivers(company):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+company
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    founded = None
    employees = None
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url in url_crunchbase:
            print(company,' in crunch base')
            try:
                firefox.get(url_crunchbase)
            except:
                pass
            
            blocks_dt = []
            blocks_dd = []
            try:
                soup = BeautifulSoup(firefox.page_source, "html.parser")
                blocks_dt = soup.select("div.details dt")
                blocks_dd = soup.select("div.details dd")
            except:
                pass
            
            i = 0
            while len(blocks_dt)==0 or len(blocks_dd)==0:
                i=i+1
                if i >= 50:
                    print('fail to crawl ', company , ' in crunchbase')
                    break
                time.sleep(0.3)
                try:
                    soup = BeautifulSoup(firefox.page_source, "html.parser")
                    blocks_dt = soup.select("div.details dt")
                    blocks_dd = soup.select("div.details dd")
                except:
                    pass
                  
            for index,block in enumerate(blocks_dt):
                if 'Founded' in  block.getText() : 
                    founded = blocks_dd[index].getText()
                if 'Employees' in block.getText():
                    employees = blocks_dd[index].getText().split('|')[0]
            print(company,founded,employees)
    return founded,employees

In [8]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
                
            if 'foundationDate' not in company and 'number_of_employees' not in company and "search_label" not in company.keys():
                i=i+1
                founded,employees = extract_company_techcrunch_2drivers(name)
                company["search_label"] = 1
                if founded is not None:
                    company["foundationDate"] = founded
                if employees is not None:
                    company["number_of_employees"] = employees
                    
            elif 'foundationDate'  in company or 'number_of_employees' in company:
                company["search_label"] = 1
                print("skip: ",name, " in crunchbase")
            
            else:
                print("skip: ",name, " not in crunchbase")
            self.save_to_disk()
                    
            
                
            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [9]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()

skip:  SENSORO  in crunchbase
skip:  Microsoft Corp  not in crunchbase
skip:  Sumitomo  not in crunchbase
skip:  Manchester City Verve  not in crunchbase
skip:  SalesWings  in crunchbase
skip:  SA tech  not in crunchbase
skip:  Avito  not in crunchbase
skip:  Tshimologong Innovation Hub  not in crunchbase
skip:  Flickr  in crunchbase
skip:  Niantic Labs  not in crunchbase
skip:  Oblong  not in crunchbase
skip:  Osterhout Design Group  not in crunchbase
skip:  Niantic  not in crunchbase
skip:  Oblong Industries  in crunchbase
skip:  Mezzanine  not in crunchbase
skip:  The Pokemon Company  in crunchbase
skip:  GamesBeat  in crunchbase
skip:  BNJ Marketing  not in crunchbase
skip:  DormaKaba  not in crunchbase
skip:  Berkshire companies  not in crunchbase
skip:  Bezos  not in crunchbase
skip:  Private Equity  not in crunchbase
skip:  Utrip  in crunchbase
skip:  Greylock  not in crunchbase
skip:  Elsight  not in crunchbase
skip:  LiveU  in crunchbase
skip:  WAN  not in crunchbase
skip:  Vo

skip:  IDEO  in crunchbase
skip:  Afrolabs  not in crunchbase
skip:  Business Partners  not in crunchbase
skip:  SMEasy  in crunchbase
skip:  Sanari  not in crunchbase
skip:  Palo Alto Software  in crunchbase
skip:  DFI  not in crunchbase
skip:  GroTech  in crunchbase
skip:  United Internet AG  in crunchbase
skip:  WumDrop  in crunchbase
skip:  NicheStreem  in crunchbase
skip:  Peacetech Accelerator  in crunchbase
skip:  PeaceTech Lab  in crunchbase
skip:  Nairobi Province Business  not in crunchbase
skip:  Spazapp  not in crunchbase
skip:  Silicon Cape  not in crunchbase
skip:  Tiger Brands  in crunchbase
skip:  Agile  in crunchbase
skip:  Strathmore  not in crunchbase
skip:  giant Visa  not in crunchbase
skip:  SME accelerator Nest  not in crunchbase
skip:  SeeBox  not in crunchbase
skip:  GetSmarter  in crunchbase
skip:  Sea Monster  in crunchbase
skip:  Clock Education  not in crunchbase
skip:  Telkom  not in crunchbase
skip:  CiTi  in crunchbase
skip:  Ogilvy & Mather  not in crun

skip:  Entersekt  in crunchbase
skip:  Arbour Group  not in crunchbase
skip:  DFS Lab  in crunchbase
skip:  iHub  in crunchbase
skip:  TFG  not in crunchbase
skip:  Social Lender  not in crunchbase
skip:  XL Embassy Travel  not in crunchbase
skip:  Holiday Factory  not in crunchbase
skip:  The Singer Group  not in crunchbase
skip:  Beachcomber  not in crunchbase
skip:  Tagmarshal  in crunchbase
skip:  Phillips Consulting Group  not in crunchbase
skip:  AIS Media  in crunchbase
skip:  Numeris Media Group  not in crunchbase
skip:  Zapphaire Events  not in crunchbase
skip:  Angel Investors  not in crunchbase
skip:  Angel  not in crunchbase
skip:  Dropbox  in crunchbase
skip:  Reddit  in crunchbase
skip:  Amref Health Africa  not in crunchbase
skip:  WePay  in crunchbase
skip:  AgroCenta  not in crunchbase
skip:  Oasis 500  in crunchbase
skip:  Sanad  not in crunchbase
skip:  IDWork  not in crunchbase
skip:  Interswitch  in crunchbase
skip:  Investment Okechukwu Enelamah  not in crunchbase

skip:  Kalani Hilliker  not in crunchbase
skip:  The Body Shop  in crunchbase
skip:  Sephora  in crunchbase
skip:  Marketo  in crunchbase
skip:  Tether Studios  not in crunchbase
skip:  Game Masons  not in crunchbase
skip:  ARPDAU  not in crunchbase
skip:  SuperData Research  in crunchbase
skip:  DataHero  in crunchbase
skip:  GE Aviation  in crunchbase
skip:  Parrable  in crunchbase
skip:  Devices Group  not in crunchbase
skip:  Docker  in crunchbase
skip:  AppExchange  not in crunchbase
skip:  Kleiner Perkins Caufield  not in crunchbase
skip:  CBRE  not in crunchbase
skip:  Invitation Homes  in crunchbase
skip:  The Blackstone Group  not in crunchbase
skip:  Viptela  in crunchbase
skip:  AppDynamics  in crunchbase
skip:  Affirmed Networks  in crunchbase
skip:  OpenDNS  in crunchbase
skip:  SDX Central  not in crunchbase
skip:  Vox Media  in crunchbase
skip:  Bloomberg Media  in crunchbase
skip:  IMG  not in crunchbase
skip:  Wendy  not in crunchbase
skip:  NBC Entertainment  not in c

skip:  Macy  not in crunchbase
skip:  Deloitte Consulting LLP  not in crunchbase
skip:  Wet Seal  not in crunchbase
skip:  Payless  not in crunchbase
skip:  The Limited  in crunchbase
skip:  car2go  in crunchbase
skip:  Moovel Group  not in crunchbase
skip:  Ars Technica  in crunchbase
skip:  Revolv  in crunchbase
skip:  Propelify Innovation  not in crunchbase
skip:  Thrive Global  in crunchbase
skip:  The Boring Company  in crunchbase
skip:  Federal Communications  not in crunchbase
skip:  DC  not in crunchbase
skip:  Oculus Research  not in crunchbase
skip:  Zipkin  not in crunchbase
skip:  Udacity  in crunchbase
skip:  Chandrasekaran  not in crunchbase
skip:  Snapdeal  in crunchbase
skip:  TaskRabbit  in crunchbase
skip:  ADP  not in crunchbase
skip:  USA Today  in crunchbase
skip:  Gusto  in crunchbase
skip:  Fidelity  not in crunchbase
skip:  Money Intelligence  not in crunchbase
skip:  Raymond James Financial Services  not in crunchbase
skip:  Apple.Zuckerberg  not in crunchbase


skip:  Viant  in crunchbase
skip:  Quattro  not in crunchbase
skip:  Belo Horizonte  not in crunchbase
skip:  Startup Farm  in crunchbase
skip:  Circuito Startup  in crunchbase
skip:  Embraco  in crunchbase
skip:  Buscape  in crunchbase
skip:  Rio  in crunchbase
skip:  Natura  not in crunchbase
skip:  Deliveroo Plus  not in crunchbase
skip:  Clipperton  in crunchbase
skip:  OVH  not in crunchbase
skip:  Oxford Nanopore  not in crunchbase
skip:  Sigfox  in crunchbase
skip:  Devialet  in crunchbase
skip:  ImageNet  in crunchbase
skip:  LibriSpeech  not in crunchbase
skip:  Tensorflow  not in crunchbase
skip:  Torch  in crunchbase
skip:  HTT  not in crunchbase
skip:  Hyperloop Transportation Technologies  in crunchbase
skip:  Abu Dhabi  not in crunchbase
skip:  Xaxis  in crunchbase
skip:  Juro  in crunchbase
skip:  Gumtree  in crunchbase
skip:  Kira Systems  in crunchbase
skip:  IAM  in crunchbase
skip:  Integrated Asset Management Limited  not in crunchbase
skip:  Commerce Sciences  in c

higi August 28, 2005 51 - 100 
High Fidelity  in crunch base
High Fidelity March 1, 2012 51 - 100 
Limitless  in crunch base
Limitless 2015 1 in Crunchbase
Bungie  in crunch base
Bungie 2015 1 in Crunchbase
GrubHub  in crunch base
GrubHub 1991 501 - 1k 
Intelius  in crunch base
Intelius 2004 12 in Crunchbase
Dwellable  in crunch base
Dwellable March 15, 2012 1k - 5k 
Versium  in crunch base
Versium March 15, 2012 1k - 5k 
TalentWise  in crunch base
TalentWise 2012 11 - 50 
MetaJure  in crunch base
MetaJure 2006 251 - 500 
MassChallenge  in crunch base
MassChallenge August 1, 2007 11 - 50 
Adhesys Medical  in crunch base
Adhesys Medical June 1, 2009 None
Socedo  in crunch base
Socedo 2013 11 - 50 
Extreme Networks  in crunch base
Extreme Networks 2013 11 - 50 
HubSpot  in crunch base
HubSpot 1996 1k - 5k 
9Mile Labs  in crunch base
9Mile Labs 1996 1k - 5k 
SheerID  in crunch base
SheerID 1996 1k - 5k 
Ivy Softworks  in crunch base
Ivy Softworks 2013 11 - 50 
CuePath Innovation  in crunc

KeyboardInterrupt: 